<a href="https://colab.research.google.com/github/sushil79g/Nepali_text_summerizer/blob/master/Nepali_summerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#if you are using colab
!git clone https://github.com/sushil79g/Nepali_text_summerizer
from Nepali_text_summerizer.loadembedding import load_vector

Cloning into 'Nepali_text_summerizer'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 15 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [0]:
#if you are at local
# from loadembedding import load_vector

In [0]:
word_vec = load_vector()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import string
def preprocess(text):
    sentences = text.split(u"।")
#     abc = text.translate(str.maketrans('', '', string.punctuation))
    sentences = [sentence.translate(str.maketrans('', '', string.punctuation)) for sentence in sentences]
    return sentences 

In [4]:
word_vec['याद']

array([ 1.59461051e-01, -5.85965931e-01,  3.78590047e-01, -4.74045545e-01,
        9.43699852e-02,  8.82810473e-01,  8.47471714e-01,  2.71366894e-01,
       -2.91820467e-01,  5.16441107e-01, -3.08772981e-01, -2.16735333e-01,
       -2.68063605e-01, -2.66613662e-01, -4.27117199e-01, -9.15338635e-01,
       -6.45890951e-01,  1.69385195e-01,  1.89498529e-01,  1.03605795e+00,
       -1.43938050e-01,  2.56531447e-01, -3.53469521e-01,  2.65364021e-01,
        3.85109991e-01,  3.01573038e-01,  7.09220588e-01, -6.60610273e-02,
        6.79583371e-01, -1.09058753e-01,  2.95875072e-01, -8.73443559e-02,
        1.30954489e-01,  4.19844911e-02,  4.76874322e-01,  5.39385259e-01,
        5.25668502e-01, -6.92974627e-02, -9.60046947e-02,  3.30410570e-01,
       -6.32967889e-01, -5.90312257e-02, -6.23273611e-01,  2.60242343e-01,
        5.40041029e-01,  2.43316591e-01,  2.72388130e-01,  4.72497255e-01,
       -1.95385918e-01, -8.09923261e-02, -8.45567584e-01, -1.58558071e-01,
       -7.79802799e-01, -

In [5]:
# (list(word_vec.vocab)) सिद्धान्त
word_vec.vector_size

400

In [0]:
#using TFIDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
def generate_centroif_tfidf(sentence):
    tf = TfidfVectorizer()
    tfidf = tf.fit_transform(sentence).toarray().sum(0)
    tfidf = np.divide(tfidf, tfidf.max())
    words = tf.get_feature_names()
    similar_term = []
    for i in range(len(tfidf)):
        if words[i] in vocab:
            if tfidf[i] >= 0.2:
                similar_term.append(words[i])
    res = [word_vec[term] for term in similar_term]
    return sum(res)/len(res)

In [0]:
vocab = word_vec.vocab
def sentence_vectorizer(sentence,size,word_vec,vocab):
    dic = {}
    for i in range(len(sentence)):
        sum_vec = np.zeros(size)
        try:
            sentence_word = sentence[i].split()
        except:
            pass
        sentence = [word for word in sentence_word if word in vocab]
        if sentence:
            for word in sentence:
                word_vec_ = word_vec[word]
                sum_vec = np.add(sum_vec, word_vec_)
                
            dic[i] = sum_vec/len(sentence)
    return dic


In [0]:
def sentence_selection(centroid, sentences_dict, summary_length):
    from scipy.spatial.distance import cosine
    sentence_retriever = []
    record = []
    for sentence_id in sentences_dict:
        vector = sentences_dict[sentence_id]
        similarity = (1 - cosine(centroid, vector))
        record.append((sentence_id, vector, similarity))

    rank = list(reversed(sorted(record, key=lambda tup: tup[2])))
    sentence_ids = []
    summary_char_num = 0
    stop = False
    i = 0
    text_length = sum([len(x) for x in sentence_retriever])

    if summary_length < 1:
        limit = int(text_length * float(summary_length))

        while not stop and i < len(rank):
            sentence_id = rank[i][0]
            new_vector = sentences_dict[sentence_id]
            sent_char_num = len(sentence_retriever[sentence_id])
            redundancy = [sentences_dict[k] for k in sentence_ids
                            if (1 - cosine(new_vector, sentences_dict[k]) > 0.95)]

            if not redundancy:
                summary_char_num += sent_char_num
                sentence_ids.append(sentence_id)
            i += 1

            if summary_char_num > limit:
                stop = True
    else: 
        sentences_number = int(summary_length)
        sentence_ids = rank[:sentences_number]
        sentence_ids = map(lambda t: t[0], sentence_ids)

    sentence_ids = sorted(sentence_ids)
    
    return sentence_ids

In [0]:
def summary(centroid_tfidf,sent,sent_dict,length):
    ids = sentence_selection(centroid_tfidf,sent_dict,length)
    whole_summary = []
    for inde in ids:
        whole_summary.append(sent[inde])
    return '।'.join(whole_summary)


In [0]:
sample_text = "मलाई राम्रैसंग याद छ, भर्खर एसएलसी सकेर बसेको थिएँ, मेरो बा रोगले लगभग थलिनु भएको थियो । त्यसको एक प्रमुख कारण हो, कच्चा बैद्यको मात्रा । प्राथमिक स्वास्थ्य केन्द पालुङका एक अहेबले लगातार पेन किलरको प्रयोगले नै मेरो बा मलगायत मेरो परिवारलाई टुहुरो बनाएर जानु भएको हो । २०५९ साल बैशाख दुई गते बाको स्वास्थ्य परिक्षणको लागि ललितपुरको पुल्चोकस्थित शान्ति फर्ममा पुग्यौं, त्यहाँ डाक्टर आनन्द कुमार श्रेष्ठले बाको बारेमा भबिष्यबाणी गरिदिनु भयो, सबै रिपोर्टहरु हेरेर । त्यस लगत्तै हामी क्यान्सरको लागि नाम कहलिएका डाक्टर मदनकुमार पियाकोमा पुग्यौं, काठमाण्डौं हस्पिटल त्रिपुरेश्वर । त्यहाँ पुग्दा मैले पियालाई चिकित्सक कम ब्यापारी बढि पाएँ । पियाको सुझाब थियो, केमोथेरापी गर्ने तर त्यसको ग्यारेन्टी थिएन । अनि उनले सोधे थैली कस्तो छ रु थैली हेरेर उपचार गर्छौं, चलाउने त केमो नै हो एकपटकको १० हजार रुपैंया पर्ने पनि छ, पैंतीस सय पर्ने पनि । तपाँईहरु सल्लाह गरेर भोलि आउनु । उनले थैली हेरे, रोग हेरेनन, रोगीको अबस्था हेरेनन । हेरे न केबल बिरामीको थैली । अनि हामीले पनि पियालाई हेरेनौं । काठमाण्डौं छाडेर हामी बिपी कोइराला क्यान्सर अस्पताल चितवन हानियौं । त्यहाँ पनि मेरो चिकित्सक, बिरामी र बिरामीको अभिभाबकबीच हुनुपर्ने जस्तो सम्बन्ध स्थापित हुन सकेन । डाक्टरले भने जति सबै प्रक्रिया र रिपोर्टहरु पूरा गरेर डाक्टर झाको टेबुलमा पुग्दा, मलाई केहि पनि थाहा छैन भन्ने जवाफ आयो, सबै रिपोर्ट हेरिसकेपछि। अनि मैले त्यहाँ औषधि पसल संचालन गरेर मामाको एक जना साथीको सोर्स फार्सले भोलिपल्ट बुबालाई अस्पताल त भर्ना गराएँ । तर, हाम्रा सबै प्रयासहरु असफल साबित भए । डाक्टर श्रेष्ठ र ती औषधीपसल संचालकले गरेको भबिष्यबाणी अनुसार नै त्यही बर्षको साउन ३० गते मेरो बा हामी सबैलाई छाडेर सदाका लागि बिदा हुनु भयो ।"

In [0]:
def show_summary(sample_text,length_sentence_predict):
    sent = preprocess(sample_text)
    centroid_tfidf = generate_centroif_tfidf(sent)
    size = word_vec.vector_size
    sent_dict = sentence_vectorizer(sent,size,word_vec,vocab)
    return summary(centroid_tfidf,sent,sent_dict,length_sentence_predict)

In [68]:
from pprint import pprint
pprint('original')
pprint(sample_text)
print('\n\n')
pprint('summerized')
pprint(show_summary(sample_text,5))

'original'
('मलाई राम्रैसंग याद छ, भर्खर एसएलसी सकेर बसेको थिएँ, मेरो बा रोगले लगभग थलिनु '
 'भएको थियो । त्यसको एक प्रमुख कारण हो, कच्चा बैद्यको मात्रा । प्राथमिक '
 'स्वास्थ्य केन्द पालुङका एक अहेबले लगातार पेन किलरको प्रयोगले नै मेरो बा '
 'मलगायत मेरो परिवारलाई टुहुरो बनाएर जानु भएको हो । २०५९ साल बैशाख दुई गते '
 'बाको स्वास्थ्य परिक्षणको लागि ललितपुरको पुल्चोकस्थित शान्ति फर्ममा पुग्यौं, '
 'त्यहाँ डाक्टर आनन्द कुमार श्रेष्ठले बाको बारेमा भबिष्यबाणी गरिदिनु भयो, सबै '
 'रिपोर्टहरु हेरेर । त्यस लगत्तै हामी क्यान्सरको लागि नाम कहलिएका डाक्टर '
 'मदनकुमार पियाकोमा पुग्यौं, काठमाण्डौं हस्पिटल त्रिपुरेश्वर । त्यहाँ पुग्दा '
 'मैले पियालाई चिकित्सक कम ब्यापारी बढि पाएँ । पियाको सुझाब थियो, केमोथेरापी '
 'गर्ने तर त्यसको ग्यारेन्टी थिएन । अनि उनले सोधे थैली कस्तो छ रु थैली हेरेर '
 'उपचार गर्छौं, चलाउने त केमो नै हो एकपटकको १० हजार रुपैंया पर्ने पनि छ, '
 'पैंतीस सय पर्ने पनि । तपाँईहरु सल्लाह गरेर भोलि आउनु । उनले थैली हेरे, रोग '
 'हेरेनन, रोगीको अबस्था हेरेनन । हेरे न केबल बिरामीको थ